Реалізація алгоритму колабораційної фільтрації на основі схожості між користувачами (User-Based Collaborative Filtering)

Завантаження даних

In [1]:
import scipy.io

# Завантаження даних про фільми
movie_ids_path = 'movies/movie_ids.txt'
with open(movie_ids_path, 'r', encoding='latin-1') as file:
    movie_names = file.readlines()

# Парсинг назв фільмів
movie_list = [line.strip().split(' ', 1)[1] for line in movie_names if len(line.strip().split(' ', 1)) == 2]

# Завантаження даних рейтингів
ratings_mat = scipy.io.loadmat('movies/movies.mat')

# Виведення інформації про завантажені дані
movie_list[:5], {key: type(value) for key, value in ratings_mat.items()}


(['Toy Story (1995)',
  'GoldenEye (1995)',
  'Four Rooms (1995)',
  'Get Shorty (1995)',
  'Copycat (1995)'],
 {'__header__': bytes,
  '__version__': str,
  '__globals__': list,
  'Y': numpy.ndarray,
  'R': numpy.ndarray})

Нормалізація Рейтингів

In [2]:
import numpy as np

# Витягнемо матриці Y та R з завантажених даних
Y = ratings_mat['Y']  # Матриця рейтингів (фільми x користувачі)
R = ratings_mat['R']  # Матриця індикаторів (фільми x користувачі)

# Нормалізація рейтингів (віднімання середнього рейтингу кожного фільму)
Y_mean = np.sum(Y, axis=1) / np.sum(R, axis=1)
Y_norm = Y - Y_mean[:, np.newaxis] * R  # Віднімаємо середній рейтинг тільки для оцінених фільмів

Обчислення косинусної схожості між користувачами

In [3]:
def cosine_similarity(ratings, epsilon=1e-9):
    # Додаємо невелике epsilon до знаменника для уникнення ділення на 0
    sim = ratings.dot(ratings.T) + epsilon
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / norms / norms.T)

# Обчислюємо косинусну схожість
user_similarity = cosine_similarity(Y_norm.T)
user_similarity.shape


(943, 943)

Прогнозування Рейтингів

In [4]:
def predict_ratings(similarity, ratings):
    # Виконуємо множення матриці рейтингів на транспоновану матрицю схожості
    weighted_sum = np.dot(ratings, similarity.T)
    
    # Обчислюємо суму абсолютних значень схожостей для кожного фільму    
    sim_sum = np.abs(similarity).sum(axis=1)
    
    # Для нормалізації ми ділимо кожен зважений рейтинг на суму схожостей    
    pred_ratings = weighted_sum / sim_sum
    
    # Додавання середнього рейтингу до прогнозованих рейтингів
    pred_ratings += Y_mean[:, np.newaxis]
    
    return pred_ratings

predicted_ratings = predict_ratings(user_similarity, Y_norm)


Отримання ТОП-5 рекомендацій для обраного користувача

In [5]:
user_id = 0 # користувач №1
user_ratings = predicted_ratings[:, user_id]  # Витягуємо прогнози для користувача №1

# Отримуємо індекси топ-5 рейтингів
top_5_indices = np.argsort(user_ratings)[-5:][::-1]

# Виводимо назви топ-5 рекомендованих фільмів для користувача №1
top_5_movies = [movie_list[index] for index in top_5_indices]
print("Топ-5 рекомендованих фільмів для користувача №1:")
for movie in top_5_movies:
    print(movie)

Топ-5 рекомендованих фільмів для користувача №1:
Entertaining Angels: The Dorothy Day Story (1996)
They Made Me a Criminal (1939)
Marlene Dietrich: Shadow and Light (1996)
Someone Else's America (1995)
Star Kid (1997)
